In [1]:
import random
import time
from datetime import datetime, timedelta
from pathlib import Path

import polars as pl
from deltalake import DeltaTable

PATH = Path("./my-test-timelake")

In [3]:
def create_sample_data(
    num_rows: int = 100_000_0,
    asset_ids: list[str] = ["AAPL", "MSFT", "GOOG", "TSLA"],
    start_date: datetime = datetime(2023, 1, 1),
) -> pl.DataFrame:
    assert num_rows >= len(asset_ids), (
        "num_rows must be at least equal to the number of asset_ids"
    )
    rows_per_asset = num_rows // len(asset_ids)
    all_data = []

    for asset_id in asset_ids:
        dates = [start_date + timedelta(hours=i) for i in range(rows_per_asset)]
        prices = [
            round(100 + i * 0.05 + (i % 24) * 0.3 + random.uniform(-1, 1), 2)
            for i in range(rows_per_asset)
        ]
        volumes = [
            int(1000 + i * 2 + (i % 10) * 50 + random.randint(-20, 20))
            for i in range(rows_per_asset)
        ]
        asset_col = [asset_id] * rows_per_asset

        all_data.append(
            pl.DataFrame(
                {
                    "date": dates,
                    "asset_id": asset_col,
                    "price": prices,
                    "volume": volumes,
                }
            )
        )

    return pl.concat(all_data)


In [4]:
df = create_sample_data()
df.head(5)

date,asset_id,price,volume
datetime[μs],str,f64,i64
2023-01-01 00:00:00,"""AAPL""",99.12,1013
2023-01-01 01:00:00,"""AAPL""",100.11,1038
2023-01-01 02:00:00,"""AAPL""",99.86,1084
2023-01-01 03:00:00,"""AAPL""",100.8,1144
2023-01-01 04:00:00,"""AAPL""",100.54,1193


In [6]:
df.shape

(1000000, 4)

### We test a little bit the polars code


In [10]:
df.write_delta(
    target=PATH,
    mode="overwrite",
)

In [11]:
dt = DeltaTable(PATH)

In [12]:
# We can open it in data wrangler!!
dt

DeltaTable()

In [13]:
dt.schema()

Schema([Field(date, PrimitiveType("timestamp_ntz"), nullable=True), Field(asset_id, PrimitiveType("string"), nullable=True), Field(price, PrimitiveType("double"), nullable=True), Field(volume, PrimitiveType("long"), nullable=True)])

In [14]:
dt.partitions()

[]

In [15]:
start = time.time()
pl.read_delta(dt)
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.022 seconds


In [16]:
dt.optimize.compact()

{'numFilesAdded': 1,
 'numFilesRemoved': 4,
 'filesAdded': '{"avg":9198633.0,"max":9198633,"min":9198633,"totalFiles":1,"totalSize":9198633}',
 'filesRemoved': '{"avg":4473232.25,"max":4473702,"min":4472159,"totalFiles":4,"totalSize":17892929}',
 'partitionsOptimized': 1,
 'numBatches': 980,
 'totalConsideredFiles': 4,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

In [17]:
start = time.time()
pl.read_delta(dt)
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.021 seconds


In [18]:
start = time.time()
pl.read_delta(dt, columns="date")
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.014 seconds


In [19]:
# When we don't push down the filtering
start = time.time()
df = pl.read_delta(dt)
df.filter(df["date"].cast(pl.Date) == pl.lit("2023-01-01").cast(pl.Date))
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.024 seconds


In [20]:
# This does not seem to work, perhaps because we have no partitioning
start = time.time()
pl.read_delta(
    dt,
    delta_table_options={
        "predicate": [
            pl.col("date").cast(pl.Date) == pl.lit("2023-01-01").cast(pl.Date)
        ]
    },
)
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.048 seconds


/Users/marcusgarsdal/Personal/timelake/.venv/lib/python3.10/site-packages/polars/io/delta.py:454: RuntimeWarning: When supplying a DeltaTable directly, `version`, `storage_options`, and `delta_table_options` are ignored.
                To silence this warning, don't supply those parameters.
  warnings.warn(


### We try a new table partitioned


In [21]:
PATH_PARTITIONED = Path("./my-test-timelake-partitioned")
df.write_delta(
    target=PATH_PARTITIONED,
    mode="overwrite",
    delta_write_options={
        "partition_by": ["asset_id"],
    },
)

dt_partitioned = DeltaTable(PATH_PARTITIONED)

In [22]:
start = time.time()
df = pl.read_delta(
    dt_partitioned,
)
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.009 seconds


In [23]:
start = time.time()
pl.from_arrow(dt_partitioned.to_pyarrow_table(partitions=[("asset_id", "=", "AAPL")]))
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.024 seconds


In [24]:
start = time.time()
df = pl.read_delta(
    dt_partitioned,
    pyarrow_options={"partitions": [("asset_id", "=", "AAPL")]},
    use_pyarrow=True,
)
end = time.time()
print(f"Read completed in {end - start:.3f} seconds")

Read completed in 0.011 seconds


In [27]:
dt.schema()

Schema([Field(date, PrimitiveType("timestamp_ntz"), nullable=True), Field(asset_id, PrimitiveType("string"), nullable=True), Field(price, PrimitiveType("double"), nullable=True), Field(volume, PrimitiveType("long"), nullable=True)])

In [30]:
dt.metadata().partition_columns

[]